In [13]:
import cv2
license_image = cv2.imread("/home/hammad/Downloads/WhatsApp Image 2022-10-04 at 12.01.39 AM.jpeg")
test_image = cv2.imread("/home/hammad/Downloads/WhatsApp Image 2022-10-04 at 12.01.59 AM.jpeg")

_, encoded_license_image = cv2.imencode('.png', license_image)
_, encoded_test_image = cv2.imencode('.png', test_image)


bytes_license_image = encoded_license_image.tobytes()
bytes_test_image = encoded_test_image.tobytes()

print(type(bytes_license_image))

<class 'bytes'>


In [1]:
!python3 face_similarity.py "/home/hammad/Downloads/WhatsApp Image 2022-10-04 at 12.01.39 AM.jpeg" "/home/hammad/Downloads/WhatsApp Image 2022-10-04 at 12.01.59 AM.jpeg"

('An Exception Occured: embedded null byte', 500)


In [1]:
from keras.models import load_model
from numpy import expand_dims
from numpy import asarray
import numpy as np
from PIL import Image
from deepface import DeepFace # pip install deepface
from retinaface import RetinaFace # pip install retina-face
# from mtcnn.mtcnn import MTCNN # pip install mtcnn

2022-10-05 17:55:55.665075: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 17:55:55.854804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-05 17:55:55.854832: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-05 17:55:55.893147: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-05 17:55:57.140662: W tensorflow/stream_executor/platform/de

In [2]:
def getModel():
    # return load_model('facenet_keras.h5')
	return DeepFace.build_model("Facenet")

def detect_face(image_array):
	face_results = RetinaFace.detect_faces(image_array)
	print("Number of faces found from Image: " +str(len(face_results)))
	return face_results

def get_embedding(face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = getModel().predict(samples)
    return yhat[0]

# extract a single face from a given photograph
def extract_face(image_array, face_box, required_size=(160, 160)):
	# extract the bounding box from the first face
	x1, y1, width, height = face_box
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = image_array[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)

	target_embedding = get_embedding(face_array)
	return target_embedding

In [39]:
ham_act = '/home/hammad/Downloads/Hammad.jpg'
ham_lic = '/home/hammad/Downloads/license3.jpg'

_lic = "/home/hammad/Downloads/WhatsApp Image 2022-10-04 at 12.01.39 AM.jpeg"
_act1 = "/home/hammad/Downloads/WhatsApp Image 2022-10-04 at 12.01.59 AM.jpeg"
_act2 = "/home/hammad/Downloads/WhatsApp Image 2022-10-04 at 12.03.23 AM.jpeg"

In [40]:
actual_image_array = np.asarray(Image.open(_act1))
license_image_array = np.asarray(Image.open(ham_lic))
print(actual_image_array.shape)
print(license_image_array.shape)

(1600, 1200, 3)
(406, 700, 3)


In [30]:
actual_embedding_list = []
actual_recognized_boxes = []

actual_face_conf = None
acutal_face_box = None

actual_face_results = detect_face(actual_image_array)
print(actual_face_results)

for actual_face in actual_face_results:
    try:
        actual_face = actual_face_results.get(actual_face)  # Retina Face
        actual_face_conf = actual_face['score']
        actual_face_box = actual_face['facial_area']
        actual_face_box = [actual_face_box[0],actual_face_box[1],actual_face_box[2]-actual_face_box[0], actual_face_box[3]-actual_face_box[1]] 
    except:
        print("Extracted face has no data.")    
        continue
    
    if actual_face_conf > 0.95:
        actual_target_embedding = extract_face(actual_image_array, actual_face_box)
        actual_embedding_list.append(actual_target_embedding)    
        actual_recognized_boxes.append(actual_face_box)

print("No. of Embeddings in actual Image: " + str(len(actual_embedding_list)))
print("#########")


license_embedding_list = []
license_recognized_boxes = []

license_face_conf = None
license_face_box = None

license_face_results = detect_face(license_image_array)
print(license_face_results)

for license_face in license_face_results:
    try:
        license_face = license_face_results.get(license_face)  # Retina Face
        license_face_conf = license_face['score']
        license_face_box = license_face['facial_area']
        license_face_box = [license_face_box[0],license_face_box[1],license_face_box[2]-license_face_box[0], license_face_box[3]-license_face_box[1]] 
    except:
        print("Extracted face has no data.")    
        continue
    
    if license_face_conf > 0.95:
        license_target_embedding = extract_face(license_image_array, license_face_box)
        license_embedding_list.append(license_target_embedding)    
        license_recognized_boxes.append(license_face_box)


print("No. of Embeddings in license Image: " + str(len(license_embedding_list)))

Number of faces found from Image: 1
{'face_1': {'score': 0.9993330836296082, 'facial_area': [1155, 1151, 1860, 2080], 'landmarks': {'right_eye': [1343.0, 1527.943], 'left_eye': [1680.0652, 1504.1787], 'nose': [1527.2783, 1691.588], 'mouth_right': [1391.7368, 1865.6737], 'mouth_left': [1662.9031, 1847.1886]}}}
1/1 [==============================] - 0s 76ms/step
No. of Embeddings in actual Image: 1
#########
Number of faces found from Image: 3
{'face_1': {'score': 0.9995424151420593, 'facial_area': [451, 244, 505, 317], 'landmarks': {'right_eye': [466.29004, 273.76425], 'left_eye': [492.0351, 274.3157], 'nose': [478.9465, 287.6408], 'mouth_right': [466.97583, 297.65808], 'mouth_left': [490.81198, 297.99496]}}, 'face_2': {'score': 0.9993719458580017, 'facial_area': [62, 128, 208, 323], 'landmarks': {'right_eye': [98.805, 200.3082], 'left_eye': [168.17332, 200.96437], 'nose': [131.01694, 234.30092], 'mouth_right': [107.446396, 275.74515], 'mouth_left': [158.6073, 275.91513]}}, 'face_3': {'

In [32]:
def distance(list1, list2):
    """Distance between two vectors."""
    squares = [(p-q) ** 2 for p, q in zip(list1, list2)]
    return sum(squares) ** .5


distances = []
for ac_emb in actual_embedding_list:
    for li_emb in license_embedding_list:  
        distances.append(distance(ac_emb, li_emb))

In [33]:
print(distances)

[16.369561071261966, 6.049696774019564, 13.100216410626743]


In [36]:
import cv2
def plotAnnotation(_boxes, image):
    # plot annotation
    annotated_image = image
    for box in _boxes:
        # annotated_image = cv2.rectangle(annotated_image, (box[0], box[1]), (box[2]+box[0], box[3]+box[1]), (255,0,0), 3) 
        annotated_image = cv2.rectangle(annotated_image, (box[0], box[1]), (box[2], box[3]), (255,0,0), 3) 
        # annotated_image = cv2.putText(annotated_image, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 2)
    return annotated_image

# ann_image = plotAnnotation([[62, 128, 208, 323]], license_image_array)
ann_image = plotAnnotation([[1155, 1151, 1860, 2080]], actual_image_array)


In [37]:
cv2.imshow("", ann_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [69]:
import re
import requests
import json
ocr_api_key = 'K83433112388957'
ocr_lang = 'eng'
ocr_overlay = 'true'


def plate_preproces(self, plate):
    plate = ''.join(e for e in plate if e.isalnum()) # remove special characters
    
    plate = re.sub('(\d+(\.\d+)?)', r' \1 ', plate).strip() # space between text and number
    

    #plate = re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', plate).strip() # space before small and capital letters

    # string to list to check words
    splits = plate.split()
    if len(splits) > 3:
        for word in splits:
            if not (re.search('^[0-9]+$', word)) and len(word) > 4: # if word is not number
                if word[0].isupper() and word[-1].islower(): # word start with upper and end lower case, removed from string
                    plate = plate.replace(word, "").strip()
    
    return plate


def img_ocr(img_path):        
    ## To read image_path for ocr
    payload = {'isOverlayRequired': ocr_overlay,
        'apikey': ocr_api_key,
        'language': ocr_lang,
        }

    with open(img_path, 'rb') as f:
        r = requests.post('https://api.ocr.space/parse/image',
            files={img_path: f},
            data=payload,
            )

    con = json.loads(r.text)

    return con['ParsedResults'][0]['ParsedText']


# im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
# # enhance image
# im1 = cv2.detailEnhance(im, sigma_s=25, sigma_r=0.15)
# # set threshold on image
# _ , thresh1 = cv2.threshold(im1, 60, 255, cv2.THRESH_BINARY)

# OCR
plate = img_ocr('/home/hammad/Downloads/license3.jpg').strip()
print(plate)

# kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
# im2 = cv2.filter2D(src=im1, ddepth=-1, kernel=kernel)






California
DRIVER LICENSE
u 11234568
08/31/201
END NONE
LNCARDHOLDER
FNIMA
257024TH TREET
ANYTOWN, CA 95818
31977
lb Iks
DD OOIOO/OOOONNNAN/ANFD/YY
08/31/2009


In [76]:
len(plate)
# mystr = '\t'.join([line.strip() for line in plate])
mystr = plate.replace("\r", "\t")
mystr = mystr.replace("\t", " ")
mystr = mystr.replace("\n", "")
print(mystr)

license_number = None
word_list = mystr.split() # string to list

for word in word_list:
    if (re.search('^[0-9]+$', word)) and len(word) == 8: # if word is a number
        license_number = word

print(license_number)

California DRIVER LICENSE u 11234568 08/31/201 END NONE LNCARDHOLDER FNIMA 257024TH TREET ANYTOWN, CA 95818 31977 lb Iks DD OOIOO/OOOONNNAN/ANFD/YY 08/31/2009
11234568


In [11]:
for dis in distances:
    if dis <= 7:
        print(dis)

6.049696774019564


In [42]:
# read 1st image and store encodings
import face_recognition # pip install face-recognition
image = cv2.imread(_lic)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# boxes = face_recognition.face_locations(rgb, model=args["detection_method"])
boxes = face_recognition.face_locations(rgb)
encodings1 = face_recognition.face_encodings(rgb, boxes)

# read 2nd image and store encodings
image = cv2.imread(_act1)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

boxes = face_recognition.face_locations(rgb)
encodings2 = face_recognition.face_encodings(rgb, boxes)

In [45]:
len(encodings1)

1

In [58]:
# now you can compare two encodings
# optionally you can pass threshold, by default it is 0.6
matches = face_recognition.compare_faces(encodings1, encodings2[0])
matches

[True]

In [52]:
import face_recognition
known_image = face_recognition.load_image_file(ham_act)
unknown_image = face_recognition.load_image_file(_act2)

biden_encoding = face_recognition.face_encodings(known_image)[0]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]

results = face_recognition.compare_faces([biden_encoding], unknown_encoding)

In [56]:
len(unknown_encoding)

128

In [53]:
results

[False]

In [63]:
#use pickle to save data into a file for later use
data = {"encodings": list_of_embeddings, "names": list_of_names}

import pickle
f = open("encodings1", "wb")
f.write(pickle.dumps(data))
f.close()

data = pickle.loads(open("encodings1", "rb").read())

In [62]:
data

array([-0.09894553,  0.03710539,  0.12140112, -0.06968689, -0.04618154,
       -0.02325607,  0.02554273,  0.00117452,  0.12625618, -0.06539699,
        0.24817342, -0.03510082, -0.17853895, -0.12592055,  0.06113213,
        0.09943514, -0.07800653, -0.11726709, -0.04366607, -0.13603181,
        0.00731657, -0.01026938, -0.02889294,  0.01736676, -0.19926412,
       -0.30530533, -0.05118365, -0.09217598,  0.01016703, -0.11577142,
       -0.01402129,  0.06626892, -0.21881552, -0.07556601, -0.05989213,
        0.11613798,  0.01260524, -0.05877529,  0.09666663,  0.02629403,
       -0.08538578, -0.08446164,  0.03968419,  0.26120317,  0.16975442,
        0.06220271,  0.04370676,  0.05502441,  0.03422422, -0.23578916,
        0.06754594,  0.05688101,  0.11091635,  0.04913695,  0.01482866,
       -0.1057803 ,  0.0305656 ,  0.01871623, -0.17208129,  0.02376074,
        0.04391514, -0.05559644, -0.0513586 , -0.00323782,  0.34716722,
        0.07585101, -0.08871283, -0.07271168,  0.19027714, -0.09

In [66]:
for val in [True,False]:
    print(val)

True
False


In [68]:
if True in [val for val in [False,True]]:
    print({"license_matched" : True})
else:
    print({"license_matched" : False})

{'license_matched': True}


In [ ]:
from __future__ import print_function, unicode_literals
from facepplib import FacePP, exceptions # pip install python-facepp
import emoji # pip install emoji
  
   
# define global variables
face_detection = ""
faceset_initialize = ""
face_search = ""
face_landmarks = ""
dense_facial_landmarks = ""
face_attributes = ""
beauty_score_and_emotion_recognition = ""
   
# define face comparing function
def face_comparing(app, Image1, Image2):
      
    print()
    print('-'*30)
    print('Comparing Photographs......')
    print('-'*30)
  
   
    cmp_ = app.compare.get(image_url1 = Image1,
                           image_url2 = Image2)
   
    print('Photo1', '=', cmp_.image1)
    print('Photo2', '=', cmp_.image2)
   
    # Comparing Photos
    if cmp_.confidence > 70:
        print('Both photographs are of same person......')
    else:
        print('Both photographs are of two different persons......')
  
          
# Driver Code 
if __name__ == '__main__':
   
    # api details
    api_key ='xQLsTmMyqp1L2MIt7M3l0h-cQiy0Dwhl'
    api_secret ='TyBSGw8NBEP9Tbhv_JbQM18mIlorY6-D'
   
    try:
   
        # create a logo of app by using iteration,
        # unicode and emoji module-------------
        for i in range(1,6):
              
            for j in range(6,-i):
                print(" " , end = " ")
                  
            for j in range(1,i):
                print('\U0001F600', end =" ")
                  
            for j in range(i,0,-1):
                print('\U0001F6A3', end= " ")
                  
            for j in range(i,1,-2):
                print('\U0001F62B', end= " ")
                  
            print()
              
        print()
   
        #print name of the app--------
        print("\t\t\t"+"Photo Comparing App\n")
       
        for i in range(1,6):
              
            for j in range(6,-i):
                print(" " , end = " ")
                  
            for j in range(1,i):
                print(emoji.emojize(":princess:"), end =" ")
                  
            for j in range(i,0,-1):
                print('\U0001F610', end= " ")
                  
            for j in range(i,1,-2):
                print(emoji.emojize(":baby:"), end= " ")
                  
            print()
           
        # call api
        app_ = FacePP(api_key = api_key, 
                      api_secret = api_secret)
        funcs = [
            face_detection,
            face_comparing_localphoto,
            face_comparing_websitephoto,
            faceset_initialize,
            face_search,
            face_landmarks,
            dense_facial_landmarks,
            face_attributes,
            beauty_score_and_emotion_recognition
        ]
          
        # Pair 1
        image1 = 'Image 1 link'
        image2 = 'Image 2 link'
        face_comparing(app_, image1, image2)
          
        # Pair2
        image1 = 'Image 1 link'
        image2 = 'Image 2 link'
        face_comparing(app_, image1, image2)        
   
    except exceptions.BaseFacePPError as e:
        print('Error:', e)

In [14]:
# convert each face in the train set to an embedding
from numpy import savez_compressed

Image_Embeddings = list()
Image_Embeddings = asarray(embedding_list)
print(Image_Embeddings.shape)

# convert each face in the test set to an embedding
# newTestX = list()
# for face_pixels in testX:
# 	embedding = get_embedding(model, face_pixels)
# 	newTestX.append(embedding)
# newTestX = asarray(newTestX)
# print(newTestX.shape)

# save arrays to one file in compressed format
savez_compressed('faces-embeddings.npz', Image_Embeddings, trainy, newTestX, testy)

NameError: name 'face_pixels' is not defined

In [ ]:
def distance(list1, list2):
    """Distance between two vectors."""
    squares = [(p-q) ** 2 for p, q in zip(list1, list2)]
    return sum(squares) ** .5

d2 = distance(source_emb, target_emb1)  
print(d2)

def image_distances(source_emb, target_emb):
    return [(x - y) * (x - y) for x,y in zip(source_emb, target_emb)]

In [ ]:
d = ((x - y)**2 + (x - y)^2)